This notebook demonstrates how to perform the following workflow:
* Find the stable states of a GINsim model,
* Export the model in NuSMV format,
* Add properties to be checked to the exported file,
* Use NuSMV to check if the model satisfies the properties,
* Create a simplified report of NuSMV results.

Analysis will be performed on the logical model "MODEL1611230001 - Traynard2016" of the mammalian cell cycle regulation, available on the BioModels (SBML format:http://www.ebi.ac.uk/biomodels-main/MODEL1611230001) and GINsim (GINsim format:http://ginsim.org/node/189) repositories.

![Mammalian cell cycle](http://ginsim.org/sites/default/files/Traynard_MultiLevelMamCC_Oct2015.png)

In this use case, we will directly use the model in GINsim format, obtained from the GINsim repository.

## Preparation

In [1]:
import os, os.path, time, subprocess

In [2]:
BASEDIR = os.getcwd()
OUTDIR = time.strftime("%Y%m%d%H%M%S_output")
os.mkdir(OUTDIR)

In [3]:
# I/O Files
MODELDIR = "snakemake/Models/2016_CellCycle_Traynard/"
ginsimModel = "Traynard_MultiLevel_MamCC_Apr2016.zginml"

## Find the stable states of a GINsim model

First we need to build the command to execute.

In [4]:
GINSIM = "GINsim" # available in docker image
GINSIM_FIND_STABLE_STATES = "snakemake/Scripts/ginsim_find_stable_states.py"

cmd1 = [GINSIM, "-s", GINSIM_FIND_STABLE_STATES, os.path.join(MODELDIR, ginsimModel)]
print(" ".join(cmd1))

GINsim -s snakemake/Scripts/ginsim_find_stable_states.py snakemake/Models/2016_CellCycle_Traynard/Traynard_MultiLevel_MamCC_Apr2016.zginml


Next step is to execute the script.

In [5]:
outputFileSS = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_stable-states.txt")
with open(outputFileSS, "w") as fout:
    subprocess.call(cmd1, stdout=fout)

Output is available in this file:

In [6]:
print(outputFileSS)

20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_stable-states.txt


Finally, as the file is quite small, we can open it and print its content in this notebook.

In [7]:
with open(outputFileSS, 'r') as fin:
    print(fin.read())

CycD Rb E2F CycE CycA CycB p27 Cdc20 Cdh1 UbcH10 Skp2 
0200-020100



For this model, we find only one stable state.

## Export the model in NuSMV format

First we need to build the command to execute.

In [8]:
GINSIM = "GINsim" # available in docker image
GINSIM_EXPORT_AS_NUSMV = "snakemake/Scripts/ginsim_export_as_NuSMV.py"

modelExportNuSMV = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_export.smv")

cmd2 = [GINSIM, "-s", GINSIM_EXPORT_AS_NUSMV, os.path.join(MODELDIR, ginsimModel), modelExportNuSMV]
print(" ".join(cmd2))

GINsim -s snakemake/Scripts/ginsim_export_as_NuSMV.py snakemake/Models/2016_CellCycle_Traynard/Traynard_MultiLevel_MamCC_Apr2016.zginml 20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_export.smv


Next step is to execute the script.

In [9]:
subprocess.call(cmd2)

0

Output is available in this file:

In [10]:
print(modelExportNuSMV)

20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_export.smv


## Add properties to be checked to the exported file

In [11]:
# One file containing the exported model
modelExportNuSMV = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_export.smv")
# One file containing the properties to be checked
propertiesFile = os.path.join(MODELDIR, "Traynard_MultiLevel_MamCC_Apr2016_property.smv")

# Naming the output file
nusmvModelwProp = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_model.smv")

[TODO] Explanation of the syntax of properties.

In [12]:
import fileinput
# Concatenate the property at the end of the model
with open(nusmvModelwProp, 'w') as fout:
    fin = fileinput.input(files=(modelExportNuSMV, propertiesFile))
    for line in fin:
        fout.write(line)
    fin.close()

Output is available in this file:

In [13]:
print(nusmvModelwProp)

20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_model.smv


## Use NuSMV to check if the model satisfies the properties

In [14]:
NUSMV = "NuSMV" # available in docker image
nusmvModelwProp = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_model.smv")

cmdNuSMV = [NUSMV, nusmvModelwProp]
print(" ".join(cmdNuSMV))

NuSMV 20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_model.smv


In [15]:
outputFileNuSMV = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_output.txt")
with open(outputFileNuSMV, "w") as fout:
    subprocess.call(cmdNuSMV, stdout=fout)

Output is available in this file:

In [16]:
print(outputFileNuSMV)

20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_output.txt


## Create a simplified report of NuSMV results

In [17]:
NUSMV_REPORT = "snakemake/Scripts/nusmv_report_all_tests.py"
outputFileNuSMV = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_output.txt")
reportFile = os.path.join(OUTDIR, "Traynard_MultiLevel_MamCC_Apr2016_report.txt")

cmdReport = ["python", NUSMV_REPORT, "-o", reportFile, outputFileNuSMV]
print(" ".join(cmdReport))

python snakemake/Scripts/nusmv_report_all_tests.py -o 20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_report.txt 20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_output.txt


In [18]:
subprocess.call(cmdReport)

0

Print report.

In [19]:
with open(reportFile, 'r') as fin:
    print(fin.read())

File	Specification	Outcome
20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_output.txt	!E [ (((CycE = 0 & CycA = 0) & CycB = 0) & Cycle1) U ((((CycE = 1 & CycA = 0) & CycB = 0) & Cycle1) & E [ (((CycE = 1 & CycA = 0) & CycB = 0) & Cycle1) U ((((CycE = 1 & CycA = 1) & CycB = 0) & Cycle1) & E [ (((CycE = 1 & CycA = 1) & CycB = 0) & Cycle1) U ((((CycE = 0 & CycA = 1) & CycB = 0) & Cycle1) & E [ (((CycE = 0 & CycA = 1) & CycB = 0) & Cycle1) U ((((CycE = 0 & CycA = 1) & CycB = 1) & Cycle1) & E [ (((CycE = 0 & CycA = 1) & CycB = 1) & Cycle1) U ((((CycE = 0 & CycA = 0) & CycB = 1) & Cycle1) & E [ (((CycE = 0 & CycA = 0) & CycB = 1) & Cycle1) U (((CycE = 0 & CycA = 0) & CycB = 0) & Cycle1) ] ) ] ) ] ) ] ) ] ) ] 	true
20170725074154_output/Traynard_MultiLevel_MamCC_Apr2016_output.txt	!E [ (((((((((((CycD = 1 & Rb = 1) & E2F = 0) & CycE = 0) & CycA = 0) & CycB = 0) & Cdh1 = 1) & Cdc20 = 0) & UbcH10 = 0) & p27 = 1) & Skp2 = 1) & Cycle1) U ((((((((((((CycD = 1 & Rb = 1) & E2F = 1) & CycE = 